In [1]:
import tensorflow_addons as tfa

import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils

C:\Users\Ambrose\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
#load data and tag

reviews = []
responses = []

with open('review.dat', 'rb') as f:
    rev = pickle.load(f)
with open('reply.dat', 'rb') as f:
    rep = pickle.load(f)
    
for i in range(len(rep)):
    responses.append( '<START> ' + rep[i] + ' <END>' )
    
reviews = rev

In [4]:
reviews = reviews[0:int(len(reviews)/2)]
responses = responses[0:int(len(responses)/2)]

In [5]:
#tokenize
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( reviews + responses )

VOCAB_SIZE = len( tokenizer.word_index )+1
#print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

In [6]:
# encoder_input_data
tokenized_reviews = tokenizer.texts_to_sequences( reviews )
maxlen_reviews = max( [ len(x) for x in tokenized_reviews ] )
padded_reviews = preprocessing.sequence.pad_sequences( tokenized_reviews , maxlen=maxlen_reviews , padding='post' )
encoder_input_data = np.array( padded_reviews )
print( encoder_input_data.shape , maxlen_reviews )

# decoder_input_data
tokenized_responses = tokenizer.texts_to_sequences( responses )
maxlen_responses = max( [ len(x) for x in tokenized_responses ] )
padded_responses = preprocessing.sequence.pad_sequences( tokenized_responses , maxlen=maxlen_responses , padding='post' )
decoder_input_data = np.array( padded_responses )
print( decoder_input_data.shape , maxlen_responses )

# decoder_output_data
tokenized_responses = tokenizer.texts_to_sequences( responses )
for i in range(len(tokenized_responses)) :
    tokenized_responses[i] = tokenized_responses[i][1:]
padded_responses = preprocessing.sequence.pad_sequences( tokenized_responses , maxlen=maxlen_responses , padding='post' )
onehot_responses = utils.to_categorical( padded_responses , VOCAB_SIZE )
decoder_output_data = np.array( onehot_responses )
print( decoder_output_data.shape )

(536, 961) 961
(536, 809) 809
(536, 809, 8656)


In [7]:
#model creation

encoder_inputs = tf.keras.layers.Input(shape=( maxlen_reviews , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_responses ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 961)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 809)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 961, 200)     1731200     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 809, 200)     1731200     input_2[0][0]                    
______________________________________________________________________________________________

In [8]:
#attempt larger batch size if comp can handle it (50)
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=25, epochs=150 ) 

# this method of saving should not be trusted
# reloading this model breaks system... saving weights better
model.save( 'model.h5' ) 

Epoch 1/150
22/22 [==============================] - 18s 373ms/step - loss: 0.7861
Epoch 2/150
22/22 [==============================] - 8s 374ms/step - loss: 0.7088
Epoch 3/150
22/22 [==============================] - 8s 377ms/step - loss: 0.7004
Epoch 4/150
22/22 [==============================] - 8s 375ms/step - loss: 0.6905
Epoch 5/150
22/22 [==============================] - 8s 382ms/step - loss: 0.6844
Epoch 6/150
22/22 [==============================] - 8s 385ms/step - loss: 0.6778
Epoch 7/150
22/22 [==============================] - 9s 392ms/step - loss: 0.6701
Epoch 8/150
22/22 [==============================] - 8s 385ms/step - loss: 0.6633
Epoch 9/150
22/22 [==============================] - 8s 380ms/step - loss: 0.6564
Epoch 10/150
22/22 [==============================] - 8s 384ms/step - loss: 0.6496
Epoch 11/150
22/22 [==============================] - 8s 385ms/step - loss: 0.6432
Epoch 12/150
22/22 [==============================] - 8s 377ms/step - loss: 0.6356
Epoch 13/150

22/22 [==============================] - 8s 372ms/step - loss: 0.2191
Epoch 100/150
22/22 [==============================] - 8s 374ms/step - loss: 0.2173
Epoch 101/150
22/22 [==============================] - 8s 374ms/step - loss: 0.2140
Epoch 102/150
22/22 [==============================] - 8s 379ms/step - loss: 0.2115
Epoch 103/150
22/22 [==============================] - 8s 375ms/step - loss: 0.2085
Epoch 104/150
22/22 [==============================] - 8s 374ms/step - loss: 0.2060
Epoch 105/150
22/22 [==============================] - 8s 372ms/step - loss: 0.2035
Epoch 106/150
22/22 [==============================] - 8s 374ms/step - loss: 0.1999
Epoch 107/150
22/22 [==============================] - 8s 375ms/step - loss: 0.2000
Epoch 108/150
22/22 [==============================] - 8s 375ms/step - loss: 0.1945
Epoch 109/150
22/22 [==============================] - 8s 374ms/step - loss: 0.1931
Epoch 110/150
22/22 [==============================] - 8s 379ms/step - loss: 0.1905
Epoch 

In [9]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [10]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
      try:
        tokens_list.append( tokenizer.word_index[ word ] ) 
      except:
        pass
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_reviews , padding='post')

In [ ]:
# this does not work. 

'''
# reloading model because I shut down my computer

import os

model = tf.keras.models.load_model('model.h5')
model.summary()
'''

In [65]:
import  nltk.translate.bleu_score as bleu
import nltk.translate.nist_score as nist

test_reviews = rev[int(len(rev)/2):]
test_responses = rep[int(len(rep)/2):]

In [66]:
print(test_responses[1])
print(test_reviews[1])

Dear Sherpa O.,Thank you for taking the time to write this review. We appreciate your patronage, and hope to see you again soon!Sincerely,168 Market Customer Service Team
Market 168 is a good alternative to the busier Asian markets in the surrounding area. Staffing has always been helpful and accommodating.Pricing has been adjusted to be competitive with the other markets in town. Hot deli offers items not found at other market steam tables.


In [67]:
enc_model , dec_model = make_inference_models()

for i in range(1):
    states_values = enc_model.predict( str_to_tokens( test_reviews[1] ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_responses:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 
    
    print(type(decoded_translation))
    print(decoded_translation[1:-4])
    a = decoded_translation[1:-4]
    
print("BLEU Score: ",bleu.sentence_bleu(test_responses[1][8:-4], a))
print("NIST Score: ",nist.sentence_nist(test_responses[1][8:-4], a))

<class 'str'>
hi madam b of the slowest in the past and i had the worst manager with the waitresses food i was so i needed like the complex very reasonable but i had like the service was good the issue with you were comfortable a extra fee usefulfunnycoolbusiness owner informationchris w business customer service7 8 2019hi branton we’re sorry to hear about your experience we appreciate your business and hope to see you again soon
BLEU Score:  8.912665928111369e-232
NIST Score:  0.017431457632880915


In [68]:
# now lets capture for 100 of the reviews

nScore = []
bScore = []
translation = []

for i in range(100):
    states_values = enc_model.predict( str_to_tokens( test_reviews[i] ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_responses:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 
    
    translation.append(decoded_translation[1:-4])
    bScore.append(bleu.sentence_bleu(test_responses[i][8:-4], decoded_translation[1:-4]))
    nScore.append(nist.sentence_nist(test_responses[i][8:-4], decoded_translation[1:-4]))


In [69]:
print("average:", sum(nScore) / len(nScore))
print("min:", min(nScore))
print("max:", max(nScore))

## for ref, the hw model returned the following scores on the same dataset
# avg: 0.000920
# min: 0.000000
# max: 0.091984

average: 0.054479363471922886
min: 0.006761076484072442
max: 0.9421978071143443
